In [0]:
import pandas as pd # 데이터 전처리
import numpy as np # 데이터 전처리
import random #데이터 전처리
from pandas import DataFrame #데이터 전처리
from collections import Counter #데이터 전처리

from tqdm import tqdm #시간 측정용

from sklearn.feature_extraction.text import CountVectorizer # model setting
from sklearn.model_selection import train_test_split  # model setting

from sklearn.naive_bayes import MultinomialNB  # model 관련
from sklearn.metrics import roc_auc_score  # model 성능 확인

In [3]:
from google.colab import drive

drive.mount('/content/dacon_data')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/dacon_data


In [4]:
cd /content/dacon_data/My Drive/dacon

/content/dacon_data/My Drive/dacon


In [5]:
train = pd.read_csv("train.csv") #해당 14th data의 csv 파일 중 train.csv 불러오기
train.head(2)

,id,year_month,text,smishing
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0


In [90]:
Counter(train['smishing'])

Counter({0: 277242, 1: 18703})

In [6]:
test = pd.read_csv("public_test.csv")
test.head(2)

,id,year_month,text
0,340000,2019-01,XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...
1,340001,2019-01,긴급 안내 XXX은행 가락동 지점 - 헬리오XXX 기본XXX 대출이자를 ...


In [0]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

fatal: destination path 'Mecab-ko-for-Google-Colab' already exists and is not an empty directory.


In [7]:
cd Mecab-ko-for-Google-Colab/

/content/dacon_data/My Drive/dacon/Mecab-ko-for-Google-Colab


In [8]:
! bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4MB 66.3MB/s 
     |████████████████████████████████| 2.3MB 39.3MB/s 
     |████████████████████████████████| 92kB 11.8MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2020-01-12 08:14:02--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.0, 18.205.93.2, 18.205.93.1, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.0|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz

In [0]:
random.seed(2019) #반복 수행시에도 동일한 결과 나올 수 있도록 시드 번호 지정
train_nsm_list=list(train[train['smishing']!=1].index)
train_nsmishing=random.sample(train_nsm_list, 14000 )

random.seed(2019)
train_sm_list=list(train[train['smishing']==1].index)
train_smishing=random.sample(train_sm_list, 2000 )

In [37]:
2000/16000

0.125

In [36]:
1000/15000

0.06666666666666667

In [10]:
train_xx=train.iloc[train_smishing+train_nsmishing,:].reset_index(drop=True) #under sampling으로 나온 index들로 train data 선별

train_yy=DataFrame(train['smishing'],columns=['smishing']) 
train_yyy=train_yy.iloc[train_smishing+train_nsmishing,:].reset_index(drop=True)

train_test=train.loc[:200]
train_test_y=DataFrame(columns=['smishing'])
print(train_test_y.shape)
train_test_y['smishing']=train['smishing'].loc[:200]
print(train_yyy.shape)

(0, 1)
(16000, 1)


In [0]:
test['smishing']=2 #train data와 동일한 형태 생성을 위해 임의의 숫자를 추가 #이후 스미싱 여부 확률 값으로 덮어 씌워짐
test_xx=DataFrame(test['text'])
test_yyy=DataFrame(test['smishing'])

In [0]:
from konlpy.tag import Okt  
okt=Okt()

In [13]:
train_doc = [ ( okt.pos(x), y ) for x, y in tqdm( zip( train_xx['text'], train_yyy['smishing'] ) ) ]

16000it [04:19, 61.64it/s] 


In [14]:
train_test_doc = [ ( okt.pos(x), y ) for x, y in tqdm( zip( train_test['text'], train_test_y['smishing'] ) ) ]

201it [00:02, 77.66it/s]


In [15]:
test_doc = [ ( okt.pos(x), y ) for x, y in tqdm( zip( test_xx['text'], test_yyy['smishing'] ) ) ]

1626it [00:30, 53.57it/s]


In [0]:
stopwords = ['께서','께','하여','해','거나','면','까지','도','하고','에게','혹은','and','or','따라','등','으로서','에는','과','으로','이라고','와','에','로','에서','1','2','3','4','5','6','의','나','수','!!','...','()',')(','.)','에도','..','!-','null','zx','!!(','))','!!!','?()','ㄹ ㅣ','.?',':?','!.',').(',')------------','.---------------','.------------','------------','----------------------------------------------','쓰시','-------------------------','!!!-','):','.-------------',')---------------------------------','----------------------------------------','ㅡㅡㅡㅡㅡㅡㅡㅡ','!)','.--------------','..(','.()','ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ','(:','!!(', '듀최',')-------------','-----',')-?','))(',')??','.!!','(...','XX','XXX', '.', '을', '를', '이', '가', '-', '(', ')', ':', '!', '?', ')-', '.-', 'ㅡ', 'XXXXXX', '..', '.(', '은', '는'] #필요없는 단어 리스트

def get_couple(_words): #필요없는 단어들 없애는 함수
    global stopwords
    _words = [x for x in _words if x[0] not in stopwords]
    l = len(_words)
    for i in range(l):
        yield _words[i][0]

기존 stopword


In [0]:
stopwords = ['XXX', '.', '을', '를', '이', '가', '-', '(', ')', ':', '!', '?', ')-', '.-', 'ㅡ', 'XXXXXX', '..', '.(', '은', '는','수'] #필요없는 단어 리스트

def get_couple(_words): #필요없는 단어들 없애는 함수
    global stopwords
    _words = [x for x in _words if x[0] not in stopwords]
    l = len(_words)
    for i in range(l-1):
        yield _words[i][0], _words[i+1][0]

In [0]:
stopwords = ['!!','...','()',')(','.)','에도','..','!-','null','zx','!!(','))','!!!','?()','ㄹ ㅣ','.?',':?','!.',').(',')------------','.---------------','.------------','------------','----------------------------------------------','쓰시','-------------------------','!!!-','):','.-------------',')---------------------------------','----------------------------------------','ㅡㅡㅡㅡㅡㅡㅡㅡ','!)','.--------------','..(','.()','ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ','(:','!!(', '듀최',')-------------','-----',')-?','))(',')??','.!!','(...','XX','XXX', '.', '을', '를', '이', '가', '-', '(', ')', ':', '!', '?', ')-', '.-', 'ㅡ', 'XXXXXX', '..', '.(', '은', '는'] #필요없는 단어 리스트

def get_couple(_words): #필요없는 단어들 없애는 함수
    global stopwords
    _words = [x for x in _words if len(x[0])>1 and x[0] not in stopwords]
    l = len(_words)
    for i in range(l-1):
        yield _words[i][0], _words[i+1][0]

In [0]:
X_train, Y_train = [], []
for lwords in train_doc:
    Y_train.append(lwords[1])
    
    temp = []
    for x in get_couple(lwords[0]):
        temp.append(x) #수정된 부분
    
    X_train.append(" ".join(temp))

In [0]:
X_train_test, Y_train_test = [], []
for lwords in train_test_doc:
    Y_train_test.append(lwords[1])
   # print(lwords)
    temp = []
    for x in get_couple(lwords[0]):
    #    print(x)
        temp.append(x) #수정된 부분)
    
    X_train_test.append(" ".join(temp))
   # print(X_train_test)

In [0]:
X_test = []
for lwords in test_doc:
    
    temp = []
    for x in get_couple(lwords[0]):
        temp.append(x)
    
    X_test.append(" ".join(temp))

In [20]:
from keras.layers import SimpleRNN, Embedding, Dense, LSTM
from keras.models import Sequential,load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from pandas import DataFrame #데이터 전처리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random #데이터 전처리
%matplotlib inline
import seaborn as sns; sns.set()

Using TensorFlow backend.


In [21]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(X_train)
sequences=tokenizer.texts_to_sequences(X_train)
word_index=tokenizer.word_index
print("Found {0} unique words: ".format(len(word_index)))
data=pad_sequences(sequences,maxlen=22034)
print("data shape: ", data.shape)

Found 22838 unique words: 
data shape:  (16000, 22034)


In [44]:
tokenizer.fit_on_texts(X_tests)
sequences=tokenizer.texts_to_sequences(X_tests)
word_index=tokenizer.word_index
print("Found {0} unique words: ".format(len(word_index)))
datass=pad_sequences(sequences,maxlen=22034)
print("datas shape: ", datass.shape)

NameError: ignored

In [22]:
tokenizer.fit_on_texts(X_train_test)
sequences=tokenizer.texts_to_sequences(X_train_test)
word_index=tokenizer.word_index
print("Found {0} unique words: ".format(len(word_index)))
datas=pad_sequences(sequences,maxlen=22034)
print("datas shape: ", datas.shape)

Found 22982 unique words: 
datas shape:  (201, 22034)


In [23]:
tokenizer.fit_on_texts(X_test)
sequences=tokenizer.texts_to_sequences(X_test)
word_index=tokenizer.word_index
print("Found {0} unique words: ".format(len(word_index)))
testdata=pad_sequences(sequences,maxlen=22034)
print("datas shape: ", testdata.shape)

Found 26218 unique words: 
datas shape:  (1626, 22034)


In [42]:
v=CountVectorizer()

v.fit(X_train)

vec_x_train= v.transform(X_train).toarray()
print(vec_x_train.shape)
#v.fit(X_train_test)
vec_x_train_test=v.transform(X_train_test).toarray()
#vec_x_test= v.transform(X_test).toarray()

(16000, 22034)


In [24]:
cd /content/dacon_data/My Drive/dacon

/content/dacon_data/My Drive/dacon


In [0]:
from keras.layers import SimpleRNN, Embedding, Dense, LSTM, Bidirectional, Dropout
from keras.models import Sequential
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [25]:
model = Sequential()
model.add(Embedding(27000, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history_ltsm = model.fit(data, Y_train, epochs=4, batch_size=60)
model.save('./model/okt_7.h5')




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          864000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 872,353
Trainable params: 872,353
Non-trainable params: 0
_________________________________________________________________


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/4





16000/16000 [==============================] - 4498s 281ms/step - loss: 0.0643 - acc: 0.9859
Epoch 2/4
16000/16000 [==============================] - 4531s 283ms/step - loss: 0.0025 - acc: 0.9995
Epoch 3/4
16000/16000 [==============================] - 4563s 28

In [0]:
document_max_num_words=16497
num_features=17212
model = Sequential()
model.add(LSTM(int(document_max_num_words*1.5), input_shape=(document_max_num_words, num_features)))
model.add(Dropout(0.3))
model.add(Dense(num_categories))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(vec_x_train, Y_train, epochs=5, batch_size=60, validation_split=0.2)
model.save('./model/okt_2.h5')

In [0]:
nmodel=load_model('./model/okt_6.h5')

In [0]:
m1= MultinomialNB()
m1.fit(vec_x_train,Y_train)

y_train_pred1=m1.predict_proba(vec_x_test)


In [0]:
y_train_pred1[0].dtype

In [0]:
y_train_pred1=model.predict_proba(testdata)
#y_train_pred1_one= [ i[1]  for i in y_train_pred1]

In [28]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(datas,Y_train_test)[1]))

201/201 [==============================] - 13s 64ms/step

 테스트 정확도: 0.9950


In [27]:
submission = pd.read_csv("submission_제출양식.csv") #해당 14th data의 csv 파일 중 train.csv 불러오기
submission.head(2)

,id,smishing
0,340000,NaN
1,340001,NaN


In [0]:
submission['smishing'] = y_train_pred1[:]

submission.to_csv("./testresult/okt_lstm_7.csv",index=False) #현재 결과물인 output2를 구글 드라이브에 submission_test라는 이름으로 저장

In [35]:
y_train_pred1.shape

(1626, 1)

In [32]:
for i in range(len(y_train_pred1)):
  print(i,":",y_train_pred1[i],Y_train_test[i])

0 : [0.] 0
1 : [1.4901161e-07] 0
2 : [0.] 0
3 : [0.] 0
4 : [0.] 0
5 : [0.] 0
6 : [2.0861626e-07] 0
7 : [2.3841858e-07] 0
8 : [0.] 0
9 : [0.] 1
10 : [1.7881393e-07] 0
11 : [0.] 0
12 : [3.2782555e-07] 0
13 : [0.] 0
14 : [1.2516975e-06] 0
15 : [2.9802322e-07] 0
16 : [2.3841858e-07] 0
17 : [0.] 0
18 : [3.874302e-06] 0
19 : [0.] 0
20 : [0.] 0
21 : [0.] 0
22 : [0.] 0
23 : [0.] 1
24 : [8.940697e-08] 0
25 : [1.1920929e-06] 0
26 : [0.] 0
27 : [5.364418e-07] 0
28 : [1.7881393e-07] 0
29 : [0.] 0
30 : [2.9802322e-07] 0
31 : [0.] 1
32 : [2.0861626e-07] 0
33 : [1.0728836e-06] 0
34 : [0.] 0
35 : [0.] 0
36 : [1.7881393e-07] 0
37 : [0.] 0
38 : [0.] 0
39 : [2.1755695e-06] 0
40 : [0.] 1
41 : [0.] 0
42 : [3.5762787e-07] 0
43 : [2.9802322e-07] 0
44 : [2.0861626e-07] 0
45 : [2.0861626e-07] 0
46 : [0.] 0
47 : [0.] 0
48 : [0.] 0
49 : [0.] 0
50 : [0.] 1
51 : [0.] 0
52 : [0.] 0
53 : [0.] 0
54 : [0.] 0
55 : [0.] 0
56 : [0.] 0
57 : [0.] 0
58 : [2.3841858e-07] 0
59 : [0.] 0
60 : [2.0861626e-07] 0
61 : [0.] 0
62 : 

IndexError: ignored

In [82]:
testresult = pd.read_csv("./testresult/okt_lstm_1.csv") #해당 14th data의 csv 파일 중 train.csv 불러오기
testresult.head(2)

,id,smishing
0,340000,0.525899
1,340001,0.999906


In [98]:
for i in range(len(testresult2)):
  if testresult2['smishing'].loc[i] >0.98:
    testresult2['smishing'].loc[i]=1
  else:
    testresult2['smishing'].loc[i]=0


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [99]:
Counter(testresult2['smishing'])

Counter({0.0: 1505, 1.0: 121})

In [91]:
370/(1256+370)

0.22755227552275523

In [100]:
121/(121+1505)

0.07441574415744158

In [0]:
testresult.to_csv("14th_baseline_multi.csv",index=False) #현재 결과물인 output2를 구글 드라이브에 submission_test라는 이름으로 저장

In [0]:
testresult2 = pd.read_csv("14th_baseline_multi.csv") #해당 14th data의 csv 파일 중 train.csv 불러오기